In [107]:
!jt -r

Reset css and font defaults in:
/Users/jeongxoo/.jupyter/custom &
/Users/jeongxoo/Library/Jupyter/nbextensions


# 문제 1

### **문제 1) Tokenizer 생성하기**

**1-1. `preprocessing()`**

텍스트 전처리를 하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장을 토큰화한 결과로, nested list 형태입니다. ex) [['i', 'go', 'to', 'school'], ['i', 'like', 'pizza']]
- 조건 1: 입력된 문장에 대해서 소문자로의 변환과 특수문자 제거를 수행합니다.
- 조건 2: 토큰화는 white space 단위로 수행합니다.
    
    

**1-2. `fit()`**

어휘 사전을 구축하는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: 위에서 만든 `preprocessing` 함수를 이용하여 각 문장에 대해 토큰화를 수행합니다.
- 조건 2: 각각의 토큰을 정수 인덱싱 하기 위한 어휘 사전(`self.word_dict`)을 생성합니다.
    - 주어진 코드에 있는 `self.word_dict`를 활용합니다.
    

**1-3. `transform()`**

어휘 사전을 활용하여 입력 문장을 정수 인덱싱하는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output: 각 문장의 정수 인덱싱으로, nested list 형태입니다. ex) [[1, 2, 3, 4], [1, 5, 6]]
- 조건 1: 어휘 사전(`self.word_dict`)에 없는 단어는 'oov'의 index로 변환합니다.

## 함수 해석
### preprocessing()
- 화이트스페이스 토크나이징
- lower case
- 가능한 케이스
- 형태소 분석기 사용 (과제 조건에 명시되어있지 않았기 때문에 가능한 것만 리드미에서 언급)


### fit()
- 모든 단어에 대해 워드 딕셔너리 생성
- task 종료 후 fit_checker = True로 설정하여
- transform()이 작동하도록 설정
- zip() 사용 고려
- 현재는 딕셔너리 컴프리헨션 사용중

- dict.fromkeys(key, value) 이용
```
string_list = ['A','B','C']
dictionary = dict.fromkeys(string_list,0)
print(dictionary)
```
- 넘파이 고려 -> 넘파이는 배열의 모양이 이상해서 사용불가




### transform()
- word_dict이 생성되어 있어야 정수 인덱싱 가능
- 따라서 fit_checker를 통해 word_dict 생성 task의 완료 여부 확인
- task가 수행되었다면, word_dict 기반 정수 인덱싱
- task가 수행되지 않았다면, return
- 개선 방향
    - 형태소 분석기를 사용하여 각 단어를 원형으로 토크나이징한 후 word_dict 생성

### fit_transform()
- fit(), transform()의 순차 실행
- fit()
    - word_dict 생성 및 fit_check 설정
- transform()
    - fit_check에 따라 정수 인덱싱 or error

In [1]:
import re
import time
import timeit
import os
import math
import itertools
import numpy as np
from functools import reduce

In [2]:
f = open(os.path.join("test_data.txt"), "r")
text = f.read()
text = text.split(".")
text = [t.replace("\n", "").strip() for t in text]
text = text[:-2]
f.close()

In [313]:
class Tokenizer():
    def __init__(self):
        self.word_dict = {'oov': 0}
        self.fit_checker = False

    def preprocessing(self, sequences):
        sub_low = lambda x: re.sub(r"[^a-zA-Z0-9]", "", x).lower()
        result = [list(map(sub_low, s.split())) for s in sequences]
        return result
  
    def fit(self, sequences):
        self.fit_checker = False
        
        tokens = self.preprocessing(sequences)
        
        # case 1
        tokens = set(itertools.chain(*tokens))

        # case 1
#         _dict_cnt = [i + 1 for i in range(len(tokens))]
#         _dict = dict(zip(tokens, _dict_cnt))

        # case 2
        st_point = max(list(self.word_dict.values()))
        _dict = {v:(i+1) + st_point for i,v in enumerate(tokens)}

        self.word_dict = dict(_dict, **self.word_dict) 
        self.fit_checker = True
    
    def transform(self, sequences):
        result = []
        tokens = self.preprocessing(sequences)
        
        if self.fit_checker:
            find_key = lambda x: x if x in self.word_dict.keys() else "oov"
            find_value = lambda x: self.word_dict[find_key(x)]
            
            result = [list(map(find_value, t)) for t in tokens]
            
            return result
        else:
            raise Exception("Tokenizer instance is not fitted yet.")
      
    def fit_transform(self, sequences):
        self.fit(sequences)
        result = self.transform(sequences)
        return result
    

```
        tokens = set(itertools.chain.from_iterable(tokens))
```

In [129]:
tk = Tokenizer()
print(timeit.timeit("tk.fit(text)", number = 100, globals = globals()))

47.07119707899983


In [130]:
print(timeit.timeit("tk.fit(text)", number = 100, globals = globals()))

48.994798257000184


In [131]:
print(timeit.timeit("tk.fit(text)", number = 100, globals = globals()))

49.49901965800018


```
        tokens = set(itertools.chain(*tokens))
```

In [125]:
tk = Tokenizer()
print(timeit.timeit("tk.fit(text)", number = 100, globals = globals()))

49.967381474999456


In [126]:
print(timeit.timeit("tk.fit(text)", number = 100, globals = globals()))

48.184304402000635


In [127]:
print(timeit.timeit("tk.fit(text)", number = 100, globals = globals()))

50.26793359000021


```
            tokens = set(reduce(lambda x, y: x + y, tokens))

```

In [133]:
tk = Tokenizer()
print(timeit.timeit("tk.fit(text)", number = 3, globals = globals()))

236.20133821000036


In [134]:
print(timeit.timeit("tk.fit(text)", number = 3, globals = globals()))

236.88421295399985


In [135]:
print(timeit.timeit("tk.fit(text)", number = 3, globals = globals()))

236.2097810060004


```
_dict = {v:i+1 for i,v in enumerate(tokens)}
```

In [167]:
tk = Tokenizer()
print(timeit.timeit("tk.fit(text)", number = 100, globals = globals()))

48.522590425998715


In [168]:
print(timeit.timeit("tk.fit(text)", number = 100, globals = globals()))

48.302762080998946


In [169]:
print(timeit.timeit("tk.fit(text)", number = 100, globals = globals()))

48.067274122000526


```
_dict_cnt = [i + 1 for i in range(len(tokens))]
_dict = dict(zip(tokens, _dict_cnt))
```

In [163]:
tk = Tokenizer()
print(timeit.timeit("tk.fit(text)", number = 100, globals = globals()))

48.305995388000156


In [164]:
print(timeit.timeit("tk.fit(text)", number = 100, globals = globals()))

48.549032615001124


In [165]:
print(timeit.timeit("tk.fit(text)", number = 100, globals = globals()))

50.83507950999956


## 테스트용 데이터 개수

In [7]:
len(text)

22399

### 1차 테스트

```
result = [list(map(lambda x: x.lower, list(filter(str.isalnum, s)))) for s in sequences]
측정결과
```

In [46]:
print(timeit.timeit("tk.preprocessing(text)", number = 100, globals = globals()))

47.13892180200003


In [54]:
print(timeit.timeit("tk.preprocessing(text)", number = 100, globals = globals()))

52.84384636100003


In [55]:
print(timeit.timeit("tk.preprocessing(text)", number = 100, globals = globals()))

55.08498289099998


## 테스트용 데이터 개수

In [8]:
len(text)

22399

### 2차 테스트

```
result = [list(map(lambda x: re.sub(r"[^a-zA-Z0-9]", "", x).lower(), s.split())) for s in sequences]
측정결과
```

In [49]:
print(timeit.timeit("tk.preprocessing(text)", number = 100, globals = globals()))

46.68618486799994


In [50]:
print(timeit.timeit("tk.preprocessing(text)", number = 100, globals = globals()))

46.044132450999996


In [51]:
print(timeit.timeit("tk.preprocessing(text)", number = 100, globals = globals()))

46.38033743599999


In [170]:
tk.fit_transform(['I go to school.', 'I LIKE pizza!'])

[[3, 5, 4, 1], [3, 2, 6]]

# 문제 2

**2-1. `fit()`**

입력 문장들을 이용해 IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list 입니다. ex) ['I go to school.', 'I LIKE pizza!']
- 조건 1: IDF 행렬은 list 형태입니다.
    - ex) [토큰1에 대한 IDF 값, 토큰2에 대한 IDF 값, .... ]
- 조건 2: IDF 값은 아래 식을 이용해 구합니다.
    
    $$
    idf(d,t)=log_e(\frac{n}{1+df(d,t)})
    $$
    
    - $df(d,t)$ : 단어 t가 포함된 문장 d의 개수
    - $n$ : 입력된 전체 문장 개수
- 조건 3: 입력된 문장의 토큰화에는 문제 1에서 만든 Tokenizer를 사용합니다.
    
    

**2-2. `transform()`**

입력 문장들을 이용해 TF-IDF 행렬을 만드는 함수입니다.

- input: 여러 영어 문장이 포함된 list입니다. ex) ['I go to school.', 'I LIKE pizza!']
- output : nested list 형태입니다.
    
    ex) [[tf-idf(1, 1), tf-idf(1, 2), tf-idf(1, 3)], [tf-idf(2, 1), tf-idf(2, 2), tf-idf(2, 3)]]
    
    |  | 토큰1 | 토큰2 | 토큰3 |
    | --- | --- | --- | --- |
    | 문장1 | tf-idf(1,1) | tf-idf(1,2) | tf-idf(1,3) |
    | 문장2 | tf-idf(2,1) | tf-idf(2,2) | tf-idf(2,3) |
- 조건1 : 입력 문장을 이용해 TF 행렬을 만드세요.
    - $tf(d, t)$ : 문장 d에 단어 t가 나타난 횟수
- 조건2 : 문제 2-1( `fit()`)에서 만든 IDF 행렬과 아래 식을 이용해 TF-IDF 행렬을 만드세요
    
    $$
    tf-idf(d,t) = tf(d,t) \times idf(d,t)
    $$

In [335]:
class TfidfVectorizer:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.fit_checker = False
        self.idf_matrix = []
        self.tfidf_matrix = []
  
    def fit(self, sequences):
        # tokenized sentense & numberized token
        tokenized = self.tokenizer.fit_transform(sequences)
        numbered_token = tk.word_dict.values()
        
        # lambda func & df value for each token
        func_df = lambda x: sum([1 for tk in tokenized if x in tk])
        token_count = {num:func_df(num) for num in numbered_token}        
        
        # lambda func & df-matrix
        find_value = lambda x: token_count[x]
        df_matrix = [list(map(find_value, tk)) for tk in tokenized]
        
        # lambda func & idf-matrix
        n = len(sequences)
        func_idf = lambda x: math.log(n / (1 + x))
        self.idf_matrix = [list(map(func_idf, df)) for df in df_matrix]
        
        self.fit_checker = True
    
    def transform(self, sequences):
        if self.fit_checker:
            tokenized = self.tokenizer.transform(sequences)
            
            get_idf = lambda x: np.array(self.idf_matrix[x])
            func_tf = lambda y: np.array(list(map(lambda x: y.count(x), y)))
            self.tfidf_matrix = [(func_tf(tk) * get_idf(i)).tolist() for i, tk in enumerate(tokenized)]
            
            return self.tfidf_matrix
        else:
            raise Exception("TfidfVectorizer instance is not fitted yet.")

  
    def fit_transform(self, sequences):
        self.fit(sequences)
        return self.transform(sequences)

In [200]:
data = ['I go to school.', 'I LIKE pizza!']

In [336]:
from faker import Faker
fake = Faker()
text = fake.texts(nb_texts = 100)

In [337]:
tk = Tokenizer()
tfidf = TfidfVectorizer(tk)

In [338]:
tfidf.fit_transform(text)

[[3.912023005428146,
  3.2188758248682006,
  3.912023005428146,
  2.995732273553991,
  2.5257286443082556,
  3.506557897319982,
  3.506557897319982,
  3.912023005428146,
  2.995732273553991,
  3.506557897319982,
  2.995732273553991,
  2.8134107167600364,
  2.995732273553991,
  2.5257286443082556,
  2.659260036932778,
  3.506557897319982,
  2.995732273553991,
  2.8134107167600364,
  3.912023005428146,
  3.506557897319982,
  3.506557897319982,
  3.2188758248682006,
  3.2188758248682006],
 [5.626821433520073,
  3.506557897319982,
  2.995732273553991,
  2.995732273553991,
  2.8134107167600364,
  2.995732273553991,
  3.2188758248682006,
  3.506557897319982,
  2.8134107167600364,
  3.506557897319982,
  6.437751649736401,
  6.437751649736401,
  3.506557897319982,
  5.626821433520073,
  2.995732273553991,
  2.995732273553991,
  3.912023005428146,
  3.506557897319982,
  2.995732273553991,
  2.995732273553991,
  3.2188758248682006,
  2.659260036932778,
  3.2188758248682006,
  3.506557897319982],

In [339]:
tk.word_dict

{'candidate': 1,
 'through': 2,
 'west': 3,
 'answer': 4,
 'politics': 5,
 'risk': 6,
 'according': 7,
 'appear': 8,
 'give': 9,
 'here': 10,
 'game': 11,
 'school': 12,
 'serve': 13,
 'who': 14,
 'degree': 15,
 'mr': 16,
 'important': 17,
 'recent': 18,
 'similar': 19,
 'from': 20,
 'north': 21,
 'around': 22,
 'knowledge': 23,
 'score': 24,
 'son': 25,
 'much': 26,
 'should': 27,
 'resource': 28,
 'shake': 29,
 'team': 30,
 'conference': 31,
 'those': 32,
 'happy': 33,
 'mission': 34,
 'seven': 35,
 'president': 36,
 'stuff': 37,
 'field': 38,
 'get': 39,
 'factor': 40,
 'actually': 41,
 'before': 42,
 'center': 43,
 'raise': 44,
 'discussion': 45,
 'half': 46,
 'bar': 47,
 'often': 48,
 'mother': 49,
 'wait': 50,
 'ability': 51,
 'floor': 52,
 'meeting': 53,
 'body': 54,
 'our': 55,
 'not': 56,
 'minute': 57,
 'ready': 58,
 'whether': 59,
 'very': 60,
 'gun': 61,
 'himself': 62,
 'service': 63,
 'leader': 64,
 'fact': 65,
 'trouble': 66,
 'food': 67,
 'sound': 68,
 'artist': 69,
 'p

## 테스트용 모듈화

In [210]:
%%writefile test_main.py

import re
import time
import os
import math
import numpy as np
from functools import reduce

class Tokenizer():
    def __init__(self):
        self.word_dict = {'oov': 0}
        self.fit_checker = False

    def preprocessing(self, sequences):
#         return [list(map(lambda x: x.lower, list(filter(str.isalnum, s)))) for s in sequences]
        return [list(map(lambda x: re.sub(r"[^a-zA-Z0-9]", "", x).lower(), s.split())) for s in sequences]
  
    def fit(self, sequences):
        self.fit_checker = False
        tokens = self.preprocessing(sequences)
        tokens = set(reduce(lambda x, y: x + y, tokens))
        self.word_dict = dict(self.word_dict, 
                              **{v:i+1 for i,v in enumerate(tokens)}) 
        self.fit_checker = True
    
    def transform(self, sequences):
        result = []
        tokens = self.preprocessing(sequences)
        if self.fit_checker:
            result = [list(map(lambda x: self.word_dict[x if x in self.word_dict.keys() else "oov"], t)) 
                      for t in tokens]
            return result
        else:
            raise Exception("Tokenizer instance is not fitted yet.")
      
    def fit_transform(self, sequences):
        self.fit(sequences)
        result = self.transform(sequences)
        return result
    

class TfidfVectorizer:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.fit_checker = False
        self.idf_matrix = []
        self.tfidf_matrix = []
  
    def fit(self, sequences):
        tokenized = self.tokenizer.fit_transform(sequences)
        
        number_token = set(reduce(lambda x, y: x + y, tokenized))
        token_count = {nt:sum([1 if nt in t else 0 for t in tokenized]) for nt in number_token}
        df = [list(map(lambda x: token_count[x], t)) for t in tokenized]

        n = len(sequences)
        self.idf_matrix = [list(map(lambda x: math.log(n / (1 + x)), d)) for d in df]
        
        self.fit_checker = True
    
    def transform(self, sequences):
        if self.fit_checker:
            tokenized = self.tokenizer.transform(sequences)
            self.tfidf_matrix = [np.array(list(map(lambda x: t.count(x), t))) * np.array(self.idf_matrix[i]) for i, t in enumerate(tokenized)]
            return self.tfidf_matrix
        else:
            raise Exception("TfidfVectorizer instance is not fitted yet.")

  
    def fit_transform(self, sequences):
        self.fit(sequences)
        return self.transform(sequences)    

Writing test_main.py
